In [22]:
# Multi-Symbol Crypto Trading Strategy
# Trading across 8 different cryptocurrencies with $80k total capital

import requests
import pandas as pd

# Define the symbols to trade
symbols = ["BTC-USDT", "ADA-USDT", "ETH-USDT", "DOGE-USDT", "XRP-USDT", "SOL-USDT", "LTC-USDT", "BNB-USDT"]

# Portfolio setup
total_capital = 80000  # $80,000 total capital
capital_per_symbol = 10000  # $10,000 allocated per symbol
transaction_fee = 0.001   # 0.1% fee per trade
trade_amount_usd = 2000   # Always buy $2000 worth per trade

print(f"Multi-Symbol Crypto Trading Strategy")
print(f"Total Capital: ${total_capital:,.2f} | Capital per Symbol: ${capital_per_symbol:,.2f}")
print(f"Symbols: {len(symbols)} cryptocurrencies | Trade Amount: ${trade_amount_usd:,.2f} per buy")
print("Fetching data and processing...")

# Initialize portfolio for each symbol
portfolio = {}
for symbol in symbols:
    portfolio[symbol] = {
        'cash': capital_per_symbol,
        'holdings': 0,
        'trades': [],
        'executed_trades': 0
    }

# Function to fetch data for a symbol
def fetch_symbol_data(symbol):
    try:
        # Fetch distance data
        distance_response = requests.get(f"https://api-crypto-monitor.nontasbak.com/distance?symbol={symbol}&window=4320")
        distance_response.raise_for_status()
        distance_data = distance_response.json()
        
        # Fetch price data
        price_response = requests.get(f"https://api-crypto-monitor.nontasbak.com/close?symbol={symbol}&window=4320")
        price_response.raise_for_status()
        price_data = price_response.json()
        
        values = distance_data['values']
        timestamps = distance_data['timestamps']
        prices = price_data['values']
        
        # Ensure matching lengths
        min_length = min(len(values), len(prices))
        values = values[:min_length]
        timestamps = timestamps[:min_length]
        prices = prices[:min_length]
        
        return values, timestamps, prices
        
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None, None, None

# Process each symbol
for symbol in symbols:
    values, timestamps, prices = fetch_symbol_data(symbol)
    
    if values is None:
        continue
    
    # Generate trading signals
    trades = []
    previous_value = None
    
    for i, (value, timestamp, price) in enumerate(zip(values, timestamps, prices)):
        if previous_value is not None:
            if previous_value <= 0 and value > 0:
                # Buy signal
                trades.append({
                    "signal": "BUY",
                    "price": price,
                    "amount_usd": trade_amount_usd,
                    "timestamp": timestamp,
                    "distance_value": value
                })
            elif previous_value > 0 and value <= 0:
                # Sell signal
                trades.append({
                    "signal": "SELL", 
                    "price": price,
                    "sell_all": True,
                    "timestamp": timestamp,
                    "distance_value": value
                })
        previous_value = value
    
    portfolio[symbol]['trades'] = trades
    portfolio[symbol]['prices'] = prices
    portfolio[symbol]['final_price'] = prices[-1] if prices else 0

# Execute trades for each symbol
for symbol in symbols:
    if 'trades' not in portfolio[symbol]:
        continue
        
    cash = portfolio[symbol]['cash']
    holdings = portfolio[symbol]['holdings']
    trades = portfolio[symbol]['trades']
    
    for trade in trades:
        if trade['signal'] == 'BUY':
            if cash >= trade['amount_usd']:
                # Calculate crypto we can buy
                gross_crypto = trade['amount_usd'] / trade['price']
                fee_crypto = gross_crypto * transaction_fee
                net_crypto = gross_crypto - fee_crypto
                
                # Update holdings
                holdings += net_crypto
                cash -= trade['amount_usd']
                portfolio[symbol]['executed_trades'] += 1
                
        elif trade['signal'] == 'SELL':
            if holdings > 0:
                # Sell all holdings
                gross_usd = holdings * trade['price']
                fee_usd = gross_usd * transaction_fee
                net_usd = gross_usd - fee_usd
                
                # Update holdings
                cash += net_usd
                holdings = 0
                portfolio[symbol]['executed_trades'] += 1
    
    # Update final portfolio state
    portfolio[symbol]['final_cash'] = cash
    portfolio[symbol]['final_holdings'] = holdings
    
    # Calculate final value
    final_crypto_value = holdings * portfolio[symbol]['final_price']
    final_total_value = cash + final_crypto_value
    
    portfolio[symbol]['final_crypto_value'] = final_crypto_value
    portfolio[symbol]['final_total_value'] = final_total_value

print("Processing complete!\n")

# Create results DataFrame
results_data = []
buy_hold_data = []

for symbol in symbols:
    if 'final_total_value' in portfolio[symbol]:
        # Strategy results
        value = portfolio[symbol]['final_total_value']
        trades = portfolio[symbol]['executed_trades']
        return_amount = value - capital_per_symbol
        return_pct = (return_amount / capital_per_symbol) * 100
        
        results_data.append({
            'Symbol': symbol,
            'Final Value': value,
            'Return ($)': return_amount,
            'Return (%)': return_pct,
            'Trades': trades
        })
        
        # Buy and hold comparison
        if 'prices' in portfolio[symbol] and portfolio[symbol]['prices']:
            initial_price = portfolio[symbol]['prices'][0]
            final_price = portfolio[symbol]['final_price']
            
            crypto_bought = capital_per_symbol / initial_price
            buy_hold_value = crypto_bought * final_price
            buy_hold_return = buy_hold_value - capital_per_symbol
            buy_hold_pct = (buy_hold_return / capital_per_symbol) * 100
            
            buy_hold_data.append({
                'Symbol': symbol,
                'Strategy Value': value,
                'Buy&Hold Value': buy_hold_value,
                'Difference': value - buy_hold_value,
                'Strategy Return (%)': return_pct,
                'Buy&Hold Return (%)': buy_hold_pct
            })

# Create DataFrames
df_results = pd.DataFrame(results_data)
df_comparison = pd.DataFrame(buy_hold_data)

# Calculate totals
total_final_value = df_results['Final Value'].sum()
total_return = df_results['Return ($)'].sum()
total_return_pct = (total_return / total_capital) * 100
total_trades = df_results['Trades'].sum()

buy_hold_total = df_comparison['Buy&Hold Value'].sum()
strategy_vs_hold_total = total_final_value - buy_hold_total
strategy_vs_hold_pct = (strategy_vs_hold_total / buy_hold_total) * 100

print("=== STRATEGY PERFORMANCE BY SYMBOL ===")
print(df_results.to_string(index=False, formatters={
    'Final Value': '${:,.2f}'.format,
    'Return ($)': '${:,.2f}'.format,
    'Return (%)': '{:.2f}%'.format
}))

print(f"\n=== TOTAL PORTFOLIO SUMMARY ===")
summary_df = pd.DataFrame([{
    'Metric': 'Initial Capital',
    'Value': f'${total_capital:,.2f}'
}, {
    'Metric': 'Final Portfolio Value',
    'Value': f'${total_final_value:,.2f}'
}, {
    'Metric': 'Total Return',
    'Value': f'${total_return:,.2f} ({total_return_pct:.2f}%)'
}, {
    'Metric': 'Total Trades Executed',
    'Value': f'{total_trades}'
}, {
    'Metric': 'Average Trades per Symbol',
    'Value': f'{total_trades / len(symbols):.1f}'
}])

print(summary_df.to_string(index=False))

print(f"\n=== STRATEGY vs BUY & HOLD COMPARISON ===")
print(df_comparison.to_string(index=False, formatters={
    'Strategy Value': '${:,.2f}'.format,
    'Buy&Hold Value': '${:,.2f}'.format,
    'Difference': '${:,.2f}'.format,
    'Strategy Return (%)': '{:.2f}%'.format,
    'Buy&Hold Return (%)': '{:.2f}%'.format
}))

print(f"\n=== OVERALL COMPARISON ===")
comparison_summary = pd.DataFrame([{
    'Strategy': 'Distance-Based Trading',
    'Final Value': f'${total_final_value:,.2f}',
    'Return (%)': f'{total_return_pct:.2f}%'
}, {
    'Strategy': 'Buy & Hold',
    'Final Value': f'${buy_hold_total:,.2f}',
    'Return (%)': f'{((buy_hold_total - total_capital) / total_capital) * 100:.2f}%'
}, {
    'Strategy': 'Difference',
    'Final Value': f'${strategy_vs_hold_total:,.2f}',
    'Return (%)': f'{strategy_vs_hold_pct:+.2f}%'
}])

print(comparison_summary.to_string(index=False))

Multi-Symbol Crypto Trading Strategy
Total Capital: $80,000.00 | Capital per Symbol: $10,000.00
Symbols: 8 cryptocurrencies | Trade Amount: $2,000.00 per buy
Fetching data and processing...
Processing complete!

=== STRATEGY PERFORMANCE BY SYMBOL ===
   Symbol Final Value Return ($) Return (%)  Trades
 BTC-USDT   $9,943.89    $-56.11     -0.56%       8
 ADA-USDT   $9,810.16   $-189.84     -1.90%       8
 ETH-USDT   $9,883.43   $-116.57     -1.17%       8
DOGE-USDT   $9,817.24   $-182.76     -1.83%       8
 XRP-USDT   $9,832.41   $-167.59     -1.68%      10
 SOL-USDT   $9,801.67   $-198.33     -1.98%       8
 LTC-USDT   $9,954.98    $-45.02     -0.45%       6
 BNB-USDT   $9,919.31    $-80.69     -0.81%       6

=== TOTAL PORTFOLIO SUMMARY ===
                   Metric               Value
          Initial Capital          $80,000.00
    Final Portfolio Value          $78,963.10
             Total Return $-1,036.90 (-1.30%)
    Total Trades Executed                  62
Average Trades per

In [23]:
# EMA Crossover Trading Strategy
# Trading across 8 different cryptocurrencies using EMA crossover signals

import requests
import json
import pandas as pd

# Define the symbols to trade
symbols = ["BTC-USDT", "ADA-USDT", "ETH-USDT", "DOGE-USDT", "XRP-USDT", "SOL-USDT", "LTC-USDT", "BNB-USDT"]

# Portfolio setup
total_capital = 80000  # $80,000 total capital
capital_per_symbol = 10000  # $10,000 allocated per symbol
transaction_fee = 0.001   # 0.1% fee per trade
trade_amount_usd = 2000   # Always buy $2000 worth per trade

print(f"EMA Crossover Trading Strategy")
print(f"Total Capital: ${total_capital:,.2f} | Capital per Symbol: ${capital_per_symbol:,.2f}")
print(f"Symbols: {len(symbols)} cryptocurrencies | Trade Amount: ${trade_amount_usd:,.2f} per buy")
print("Strategy: Buy when short EMA crosses above long EMA, Sell when short EMA crosses below long EMA")
print("Fetching data and processing...")

# Initialize portfolio for each symbol
portfolio_ema = {}
for symbol in symbols:
    portfolio_ema[symbol] = {
        'cash': capital_per_symbol,
        'holdings': 0,
        'trades': [],
        'executed_trades': 0
    }

# Function to fetch EMA data for a symbol
def fetch_ema_data(symbol):
    try:
        # Fetch short-term EMA data
        short_response = requests.get(f"https://api-crypto-monitor.nontasbak.com/ema?symbol={symbol}&window=4320&type=short")
        short_response.raise_for_status()
        short_data = short_response.json()
        
        # Fetch long-term EMA data
        long_response = requests.get(f"https://api-crypto-monitor.nontasbak.com/ema?symbol={symbol}&window=4320&type=long")
        long_response.raise_for_status()
        long_data = long_response.json()
        
        # Fetch price data
        price_response = requests.get(f"https://api-crypto-monitor.nontasbak.com/close?symbol={symbol}&window=4320")
        price_response.raise_for_status()
        price_data = price_response.json()
        
        short_ema = short_data['values']
        long_ema = long_data['values']
        timestamps = short_data['timestamps']
        prices = price_data['values']
        
        # Ensure matching lengths
        min_length = min(len(short_ema), len(long_ema), len(prices))
        short_ema = short_ema[:min_length]
        long_ema = long_ema[:min_length]
        timestamps = timestamps[:min_length]
        prices = prices[:min_length]
        
        return short_ema, long_ema, timestamps, prices
        
    except Exception as e:
        print(f"Error fetching EMA data for {symbol}: {e}")
        return None, None, None, None

# Process each symbol
for symbol in symbols:
    short_ema, long_ema, timestamps, prices = fetch_ema_data(symbol)
    
    if short_ema is None:
        continue
    
    # Generate trading signals based on EMA crossovers
    trades = []
    previous_short = None
    previous_long = None
    
    for i, (short, long, timestamp, price) in enumerate(zip(short_ema, long_ema, timestamps, prices)):
        if previous_short is not None and previous_long is not None:
            # Check for crossover signals
            if previous_short <= previous_long and short > long:
                # Buy signal: short EMA crosses above long EMA (bullish)
                trades.append({
                    "signal": "BUY",
                    "price": price,
                    "amount_usd": trade_amount_usd,
                    "timestamp": timestamp,
                    "short_ema": short,
                    "long_ema": long
                })
            elif previous_short >= previous_long and short < long:
                # Sell signal: short EMA crosses below long EMA (bearish)
                trades.append({
                    "signal": "SELL", 
                    "price": price,
                    "sell_all": True,
                    "timestamp": timestamp,
                    "short_ema": short,
                    "long_ema": long
                })
        
        previous_short = short
        previous_long = long
    
    portfolio_ema[symbol]['trades'] = trades
    portfolio_ema[symbol]['prices'] = prices
    portfolio_ema[symbol]['final_price'] = prices[-1] if prices else 0

# Execute trades for each symbol
for symbol in symbols:
    if 'trades' not in portfolio_ema[symbol]:
        continue
        
    cash = portfolio_ema[symbol]['cash']
    holdings = portfolio_ema[symbol]['holdings']
    trades = portfolio_ema[symbol]['trades']
    
    for trade in trades:
        if trade['signal'] == 'BUY':
            if cash >= trade['amount_usd']:
                # Calculate crypto we can buy
                gross_crypto = trade['amount_usd'] / trade['price']
                fee_crypto = gross_crypto * transaction_fee
                net_crypto = gross_crypto - fee_crypto
                
                # Update holdings
                holdings += net_crypto
                cash -= trade['amount_usd']
                portfolio_ema[symbol]['executed_trades'] += 1
                
        elif trade['signal'] == 'SELL':
            if holdings > 0:
                # Sell all holdings
                gross_usd = holdings * trade['price']
                fee_usd = gross_usd * transaction_fee
                net_usd = gross_usd - fee_usd
                
                # Update holdings
                cash += net_usd
                holdings = 0
                portfolio_ema[symbol]['executed_trades'] += 1
    
    # Update final portfolio state
    portfolio_ema[symbol]['final_cash'] = cash
    portfolio_ema[symbol]['final_holdings'] = holdings
    
    # Calculate final value
    final_crypto_value = holdings * portfolio_ema[symbol]['final_price']
    final_total_value = cash + final_crypto_value
    
    portfolio_ema[symbol]['final_crypto_value'] = final_crypto_value
    portfolio_ema[symbol]['final_total_value'] = final_total_value

print("Processing complete!\n")

# Create results DataFrame
results_data_ema = []
buy_hold_data_ema = []

for symbol in symbols:
    if 'final_total_value' in portfolio_ema[symbol]:
        # Strategy results
        value = portfolio_ema[symbol]['final_total_value']
        trades = portfolio_ema[symbol]['executed_trades']
        return_amount = value - capital_per_symbol
        return_pct = (return_amount / capital_per_symbol) * 100
        
        results_data_ema.append({
            'Symbol': symbol,
            'Final Value': value,
            'Return ($)': return_amount,
            'Return (%)': return_pct,
            'Trades': trades
        })
        
        # Buy and hold comparison
        if 'prices' in portfolio_ema[symbol] and portfolio_ema[symbol]['prices']:
            initial_price = portfolio_ema[symbol]['prices'][0]
            final_price = portfolio_ema[symbol]['final_price']
            
            crypto_bought = capital_per_symbol / initial_price
            buy_hold_value = crypto_bought * final_price
            buy_hold_return = buy_hold_value - capital_per_symbol
            buy_hold_pct = (buy_hold_return / capital_per_symbol) * 100
            
            buy_hold_data_ema.append({
                'Symbol': symbol,
                'Strategy Value': value,
                'Buy&Hold Value': buy_hold_value,
                'Difference': value - buy_hold_value,
                'Strategy Return (%)': return_pct,
                'Buy&Hold Return (%)': buy_hold_pct
            })

# Create DataFrames
df_results_ema = pd.DataFrame(results_data_ema)
df_comparison_ema = pd.DataFrame(buy_hold_data_ema)

# Calculate totals
total_final_value_ema = df_results_ema['Final Value'].sum()
total_return_ema = df_results_ema['Return ($)'].sum()
total_return_pct_ema = (total_return_ema / total_capital) * 100
total_trades_ema = df_results_ema['Trades'].sum()

buy_hold_total_ema = df_comparison_ema['Buy&Hold Value'].sum()
strategy_vs_hold_total_ema = total_final_value_ema - buy_hold_total_ema
strategy_vs_hold_pct_ema = (strategy_vs_hold_total_ema / buy_hold_total_ema) * 100

print("=== EMA STRATEGY PERFORMANCE BY SYMBOL ===")
print(df_results_ema.to_string(index=False, formatters={
    'Final Value': '${:,.2f}'.format,
    'Return ($)': '${:,.2f}'.format,
    'Return (%)': '{:.2f}%'.format
}))

print(f"\n=== EMA TOTAL PORTFOLIO SUMMARY ===")
summary_df_ema = pd.DataFrame([{
    'Metric': 'Initial Capital',
    'Value': f'${total_capital:,.2f}'
}, {
    'Metric': 'Final Portfolio Value',
    'Value': f'${total_final_value_ema:,.2f}'
}, {
    'Metric': 'Total Return',
    'Value': f'${total_return_ema:,.2f} ({total_return_pct_ema:.2f}%)'
}, {
    'Metric': 'Total Trades Executed',
    'Value': f'{total_trades_ema}'
}, {
    'Metric': 'Average Trades per Symbol',
    'Value': f'{total_trades_ema / len(symbols):.1f}'
}])

print(summary_df_ema.to_string(index=False))

print(f"\n=== EMA STRATEGY vs BUY & HOLD COMPARISON ===")
print(df_comparison_ema.to_string(index=False, formatters={
    'Strategy Value': '${:,.2f}'.format,
    'Buy&Hold Value': '${:,.2f}'.format,
    'Difference': '${:,.2f}'.format,
    'Strategy Return (%)': '{:.2f}%'.format,
    'Buy&Hold Return (%)': '{:.2f}%'.format
}))

print(f"\n=== EMA OVERALL COMPARISON ===")
comparison_summary_ema = pd.DataFrame([{
    'Strategy': 'EMA Crossover Trading',
    'Final Value': f'${total_final_value_ema:,.2f}',
    'Return (%)': f'{total_return_pct_ema:.2f}%'
}, {
    'Strategy': 'Buy & Hold',
    'Final Value': f'${buy_hold_total_ema:,.2f}',
    'Return (%)': f'{((buy_hold_total_ema - total_capital) / total_capital) * 100:.2f}%'
}, {
    'Strategy': 'Difference',
    'Final Value': f'${strategy_vs_hold_total_ema:,.2f}',
    'Return (%)': f'{strategy_vs_hold_pct_ema:+.2f}%'
}])

print(comparison_summary_ema.to_string(index=False))

EMA Crossover Trading Strategy
Total Capital: $80,000.00 | Capital per Symbol: $10,000.00
Symbols: 8 cryptocurrencies | Trade Amount: $2,000.00 per buy
Strategy: Buy when short EMA crosses above long EMA, Sell when short EMA crosses below long EMA
Fetching data and processing...
Processing complete!

=== EMA STRATEGY PERFORMANCE BY SYMBOL ===
   Symbol Final Value Return ($) Return (%)  Trades
 BTC-USDT   $9,960.88    $-39.12     -0.39%       5
 ADA-USDT   $9,932.08    $-67.92     -0.68%       2
 ETH-USDT   $9,879.95   $-120.05     -1.20%       5
DOGE-USDT   $9,919.97    $-80.03     -0.80%       2
 XRP-USDT   $9,969.64    $-30.36     -0.30%       1
 SOL-USDT   $9,943.46    $-56.54     -0.57%       2
 LTC-USDT   $9,941.76    $-58.24     -0.58%       3
 BNB-USDT   $9,988.05    $-11.95     -0.12%       1

=== EMA TOTAL PORTFOLIO SUMMARY ===
                   Metric             Value
          Initial Capital        $80,000.00
    Final Portfolio Value        $79,535.80
             Total

In [24]:
# Strategy Comparison: Distance vs EMA Crossover
# Comparing the performance of both trading strategies

import pandas as pd

print("=== STRATEGY COMPARISON: DISTANCE vs EMA CROSSOVER ===\n")

# Assume we have the results from both strategies
# Distance strategy results (from previous cell)
distance_final_value = total_final_value  # From the distance strategy
distance_return = total_return
distance_return_pct = total_return_pct
distance_trades = total_executed_trades

# EMA strategy results (from previous cell)
ema_final_value = total_final_value_ema
ema_return = total_return_ema
ema_return_pct = total_return_pct_ema
ema_trades = total_trades_ema

# Create comparison DataFrame
strategy_comparison = pd.DataFrame([{
    'Strategy': 'Distance-Based',
    'Final Value': f'${distance_final_value:,.2f}',
    'Total Return': f'${distance_return:,.2f}',
    'Return (%)': f'{distance_return_pct:.2f}%',
    'Total Trades': distance_trades,
    'Avg Trades/Symbol': f'{distance_trades / len(symbols):.1f}'
}, {
    'Strategy': 'EMA Crossover',
    'Final Value': f'${ema_final_value:,.2f}',
    'Total Return': f'${ema_return:,.2f}',
    'Return (%)': f'{ema_return_pct:.2f}%',
    'Total Trades': ema_trades,
    'Avg Trades/Symbol': f'{ema_trades / len(symbols):.1f}'
}, {
    'Strategy': 'Buy & Hold',
    'Final Value': f'${buy_hold_total:,.2f}',
    'Total Return': f'${buy_hold_total - total_capital:,.2f}',
    'Return (%)': f'{((buy_hold_total - total_capital) / total_capital) * 100:.2f}%',
    'Total Trades': 0,
    'Avg Trades/Symbol': '0.0'
}])

print("Overall Strategy Performance:")
print(strategy_comparison.to_string(index=False))

# Calculate which strategy performed better
better_strategy = "Distance-Based" if distance_final_value > ema_final_value else "EMA Crossover"
value_difference = abs(distance_final_value - ema_final_value)
pct_difference = abs(distance_return_pct - ema_return_pct)

print(f"\n=== STRATEGY WINNER ===")
print(f"Best Performing Strategy: {better_strategy}")
print(f"Value Difference: ${value_difference:,.2f}")
print(f"Return Difference: {pct_difference:.2f} percentage points")

# Per-symbol comparison
print(f"\n=== PER-SYMBOL STRATEGY COMPARISON ===")
symbol_comparison_data = []

for symbol in symbols:
    distance_value = portfolio.get(symbol, {}).get('final_total_value', capital_per_symbol)
    ema_value = portfolio_ema.get(symbol, {}).get('final_total_value', capital_per_symbol)
    distance_trades_sym = portfolio.get(symbol, {}).get('executed_trades', 0)
    ema_trades_sym = portfolio_ema.get(symbol, {}).get('executed_trades', 0)
    
    distance_return_sym = distance_value - capital_per_symbol
    ema_return_sym = ema_value - capital_per_symbol
    
    better_for_symbol = "Distance" if distance_value > ema_value else "EMA"
    difference = abs(distance_value - ema_value)
    
    symbol_comparison_data.append({
        'Symbol': symbol,
        'Distance Value': f'${distance_value:,.2f}',
        'EMA Value': f'${ema_value:,.2f}',
        'Better Strategy': better_for_symbol,
        'Difference': f'${difference:,.2f}',
        'Distance Trades': distance_trades_sym,
        'EMA Trades': ema_trades_sym
    })

symbol_comparison_df = pd.DataFrame(symbol_comparison_data)
print(symbol_comparison_df.to_string(index=False))

# Summary statistics
distance_wins = sum(1 for row in symbol_comparison_data if row['Better Strategy'] == 'Distance')
ema_wins = sum(1 for row in symbol_comparison_data if row['Better Strategy'] == 'EMA')

print(f"\n=== SYMBOL-LEVEL PERFORMANCE ===")
print(f"Distance strategy wins: {distance_wins} symbols")
print(f"EMA strategy wins: {ema_wins} symbols")

# Trading frequency comparison
print(f"\n=== TRADING FREQUENCY ANALYSIS ===")
print(f"Distance strategy: {distance_trades} total trades ({distance_trades / len(symbols):.1f} avg per symbol)")
print(f"EMA strategy: {ema_trades} total trades ({ema_trades / len(symbols):.1f} avg per symbol)")
print(f"Trade frequency difference: {abs(distance_trades - ema_trades)} trades")

=== STRATEGY COMPARISON: DISTANCE vs EMA CROSSOVER ===

Overall Strategy Performance:
      Strategy Final Value Total Return Return (%)  Total Trades Avg Trades/Symbol
Distance-Based  $78,963.10   $-1,036.90     -1.30%            62               7.8
 EMA Crossover  $79,535.80     $-464.20     -0.58%            21               2.6
    Buy & Hold  $77,638.99   $-2,361.01     -2.95%             0               0.0

=== STRATEGY WINNER ===
Best Performing Strategy: EMA Crossover
Value Difference: $572.69
Return Difference: 0.72 percentage points

=== PER-SYMBOL STRATEGY COMPARISON ===
   Symbol Distance Value EMA Value Better Strategy Difference  Distance Trades  EMA Trades
 BTC-USDT      $9,943.89 $9,960.88             EMA     $16.99                8           5
 ADA-USDT      $9,810.16 $9,932.08             EMA    $121.92                8           2
 ETH-USDT      $9,883.43 $9,879.95        Distance      $3.48                8           5
DOGE-USDT      $9,817.24 $9,919.97           